In [11]:
# 라이브러리 임포트
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys
import os

os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\KOSPI200_NEWS')
corplist200 = pd.read_csv('corplist200.csv')


In [12]:
corplist2001 = []
for i in range(len(corplist200)):
    target = corplist200['code'][i]
    corplist2001.append('{0:06d}'.format(target))
corplist200['code'] = corplist2001
corplist200

,company,code
0,삼성전자,005930
1,SK하이닉스,000660
2,NAVER,035420
3,LG화학,051910
4,현대차,005380
...,...,...
147,일양약품,007570
148,JW중외제약,001060
149,SPC삼립,005610
150,삼양사,145990


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier

# 스코어 저장
scoredata = {}
columns_sc = ['codeno', 'logistic_acc', 'randomFore_acc']
scoredf = pd.DataFrame(scoredata, columns=columns_sc)

# 단어저장
clpndata = {}
columns_clpn = ['codeno', 'positive_corr10', 'negative_corr10']
clpndf = pd.DataFrame(clpndata, columns=columns_clpn)

In [14]:
### 뉴스데이터 전치리
# 개별기업 뉴스 db에서 기사 가져오기
# 기업별 뉴스 DB 모음으로 이행
# os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\newss')
def getSub(dbname):
#     dbname='yinfo_KCC.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='select content from news '
        res=cursor.execute(sql).fetchall()
        result=[]
        for r in res:
            result.append(r[0])
            
    return result

# 꼬마 불러오기
from konlpy.tag import Kkma
kkma=Kkma()

# 주요 형태소만으로 이루어진 기사만들기
def getPOS(rres,limlen=0):
    p=kkma.pos(rres)
    dictpos={}
    words=[]
    #표제어 추출
    check=['NNG','NNP','NNB','NNM','VV','VA','VXV','VXA','VCP','VCN','XR']
    for  w,pos in p:
        if(pos in check):
            if(len(w)>limlen): #불용어 처리 : 숫자,외국어, 특수문자와 광고문자, 길이가 적은 문자(1개 이하) 모두 완료.
                words.append(w)
                if (w in dictpos.keys()):
                    dictpos[w]+=1
                else:
                    dictpos[w]=1
    return (dictpos,words)  

#하나의 주요한 형태소만으로 이루어진 기사내용(list)를 하나의 글로 만듬
def getCorpus(doc):
    getp=getPOS(doc,1)
    sen=' '.join(getp[1])
    return sen

def getDB(sql='select nidx,ndate from nnews where nday is NULL'):
    with sqlite3.connect(dbname) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc'
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))

#### 3개 언론사가 kospi 기업별로 합쳐진 db를 넣으세요.
def getDB2(sql='select nidx,ndate from nnews where nday is NULL'):
    with sqlite3.connect(dbname2) as conn:
        cur=conn.cursor()
        #sql='select * from stockp where skey>20200101 order by skey asc' 
        qry=cur.execute(sql)
        desc=[row[0] for row in qry.description]
        data=qry.fetchall()
    return((desc,data))


In [ ]:
for i in range(50,51):
    codeno = str(corplist200['code'][i])
    # 뉴스파일 이름
    dbname = 'news_' + codeno +'.db'
    print("------------------------"+ corplist200['company'][i] + corplist200['code'][i]+'--------------------------')
    
    # 본격코드
    os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\newss')
    docs = getSub(dbname)
    corpus=[]
    for doc in docs:
        getp=getCorpus(doc)
        corpus.append(getp)

    sql= "select * from news"
    gdata=getDB(sql)
    df=pd.DataFrame(data=gdata[1],columns=gdata[0])
    df['content'] = corpus
    gdf = df[['title', 'date', 'content']]

    # 날짜 형식 전처리
    for i in range(len(gdf)):
        gdf['date'][i] = gdf['date'][i].strip()
        gdf['date'][i] = gdf['date'][i][:10].replace('.','-')

    ### 주식데이터 전처리
    # 기업별 주가 모음 DB로 연결
    os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\total')
    # 주식파일 이름
    dbname2 = codeno + '.db'
    
    sql= "select sdate,sopen,shigh,slow,sclose,svol from stockp "
    mdata=getDB2(sql)
    df=pd.DataFrame(data=mdata[1],columns=mdata[0])

    # df 정렬 순서 바꾸기
    df = df[::-1]
    df['ydclose']=df['sclose'].shift(+1)
    df['polarity']=df['sclose']-df['ydclose']
    for i in range(len(df)):
        if df['polarity'][i] > 0:
            df['polarity'][i] = 1
        else:
            df['polarity'][i] = 0
            
    ### 텍스트 처리

    gdf.columns = ['title', 'sdate', 'content']

    # 개별 뉴스에 극성을 붙이는 것이 목적 -> 뉴스기준으로 merge
    a = pd.merge(gdf, df, left_on='sdate', right_on='sdate', how='left')
    a = a.dropna(axis=0)
    b = a[['sdate','content','polarity']]
    b = b[::-1]
    b = b.reset_index(drop=True)

    # 다른 언론사별로 데이터가 누적되어있어서 날짜순 정렬로 바꿈
    bb = b.sort_values(by=['sdate'], axis=0).reset_index(drop=True)

    # train, test 를 x(nl),y(polarity) 고려 나눔 < train : 0.8기준 >
    
    x_data = bb['content']
    y_data = bb['polarity']
    train_num= int(len(y_data)*0.7)
#     train_num= 550
    xtr = x_data[:train_num]
    xte = x_data[train_num:]
    ytr = y_data[:train_num]
    yte = y_data[train_num:]
    tr = bb[:train_num]
    te = bb[train_num:]
#     tr = bb[:2202]
#     te = bb[2202:]
#     xtr = tr['content']
#     ytr = tr['polarity']
#     xte = te['content']
#     yte = te['polarity']


    ### tfidf 변환
    vect=TfidfVectorizer(min_df=2)
    vect.fit(xtr)
    tdm = vect.transform(xtr).toarray()
    cols=vect.get_feature_names()
    tdmdf=pd.DataFrame(data=tdm, columns=cols)
    
    ### 코사인 유사도
    test1 = vect.transform(xte).toarray()
    testdf=pd.DataFrame(data=test1, columns=cols) 
    # train과 test의 유사도 _ 결국 이부분이 예측
    from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도 
    similarity = cosine_similarity(testdf,tdmdf)

    # 목적 test기사 250개의 극성 구하기
    sim = []
    por = []
    tidx = []
    for i in range(len(similarity)):
        toppidx = np.argsort(similarity[i])[-1] # 가장 높은 1개로 함. #### 변경가능
#         print(similarity[i][toppidx])
        tidx.append(toppidx)
        sim.append(similarity[i][toppidx])
        por.append(ytr[toppidx])

    simdf = pd.DataFrame (sim,columns=['sim'])
    tidxdf = pd.DataFrame (tidx,columns=['tidx'])
    pordf = pd.DataFrame (por,columns=['por'])
    cos_f = pd.concat([pordf, simdf, tidxdf], axis=1)
    real = yte.reset_index(drop=True)
    cos_p = cos_f['por']

    print('confusion_matrix : \n', confusion_matrix(real, cos_p))
    print(' ')
    print('accuracy_score : ',accuracy_score(real, cos_p))
    tee = te.copy().reset_index()
    tee['pre_por'] = cos_f['por']

    # tee.groupby
    tee_f = tee[['sdate','pre_por','polarity']].groupby(tee['sdate']).mean()
    os.chdir('C:\\Users\\pcsea\\work1\\Self_study\\Company_news_PJ\\KOSPI200_NEWS\\nlp_cos')
    ### 여기는 종목별로 csv만들어야
    #1 종목별 날짜의 '코사인 유사도 활용 예측극성'과 '실제 극성' 
    t1 = tee_f.reset_index(drop=False)
    print(t1)
    t1.to_csv(codeno +'_cos.csv', encoding='cp949', index=False)
    


    ### tfidf기반 nlp 머신러닝으로 주가 예측

    #1. 로지스틱 회귀
    lgclf = LogisticRegression(random_state=0)
    params = { "C": [0.1,1,10,100]}
    grid_cv = GridSearchCV(lgclf, param_grid=params, cv=5, scoring='accuracy', verbose=1 )
    grid_cv.fit(tdm ,ytr)
    print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

    from sklearn.metrics import accuracy_score
    test = vect.transform(xte)
    best_estimator = grid_cv.best_estimator_
    preds = best_estimator.predict(test)
    lr_acscore = accuracy_score(preds,yte)
    print('logistict acc:', accuracy_score(preds,yte))

    #2. 
    # RandomForestClassifier
    from sklearn.ensemble import RandomForestClassifier
    rp = RandomForestClassifier()
    params = { "n_estimators": [10,1,2,3,4,5,6,7,8,9]}
    grid_cv = GridSearchCV(rp, param_grid=params, cv=3, scoring='accuracy', verbose=1 )
    rpr = grid_cv.fit(tdm ,ytr)
    print(grid_cv.best_params_, round(grid_cv.best_score_, 4))

    from sklearn.metrics import accuracy_score
    test = vect.transform(xte)
    best_estimator = grid_cv.best_estimator_
    preds = best_estimator.predict(test)
    rf_acscore = accuracy_score(preds,yte)
    print('RamdomForest acc:', accuracy_score(preds,yte))

    ### 단어의 정확도 파악
    new_data = {
        'codeno': codeno,
        'logistic_acc': lr_acscore,
        'randomFore_acc': rf_acscore
    }
    scoredf = scoredf.append(new_data, ignore_index=True)
    scoredf
    
    # # 주식설명 2                  ############ 요부분 오래걸림
    dp = bb[['sdate','polarity']][:train_num]
    tidfnew = pd.concat([dp, tdmdf],axis =1,ignore_index=True)
    tidfnew.columns = ['sdate','polarity'] + cols

    df = pd.DataFrame(tidfnew)
    # df.columns = ['sdate','polarity'] + cols
    corr = df.corr(method = 'pearson')
    print('양의 상관관계 :', corr['polarity'].sort_values()[-11:-1])
    positive_corr10 = corr['polarity'].sort_values()[-11:-1]
    print('음의 상관관계 :', corr['polarity'].sort_values()[:10])
    negative_corr10 = corr['polarity'].sort_values()[:10]

    pdf = positive_corr10.reset_index(drop=False)
    ndf = negative_corr10.reset_index(drop=False)


    pdf.to_csv(codeno +'_pcorr_df.csv', encoding='cp949', index=False)
    pdf.to_csv(codeno +'_ncorr_df.csv', encoding='cp949', index=False)
    
scoredf.to_csv(codeno +'scoredf.csv', encoding='cp949', index=False)

------------------------SK하이닉스000660--------------------------


C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\IPython\core\interactiveshell.py:3437: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0.5304888308003705
0.8104533899676708
0.2895928636152439
0.29801885945399487
0.27099678354992784
0.4936063205294939
0.45561474537946545
0.8068961996520743
0.5502766453710228
0.8480797764246646
0.8376912604999789
0.7411007356361485
0.5582668586663939
0.3446346913995073
0.7598947776069812
0.3944145976001868
0.7815420847290885
0.25880878070705376
0.8759882511948326
0.40900030210984084
0.33023629308943253
0.7254660904368556
0.8611771702075773
0.6188578908038197
0.41894021258982844
1.0
0.4701845033283118
0.9410074793815796
0.5703843765855127
0.5061314077158361
0.4473702713798826
0.21297607724728648
0.4072394592544727
0.773462890352723
0.8554339283978138
0.324233285002108
0.43716205259501884
0.4022316089024661
0.5120158723809353
0.5935557680247997
0.5130118635040601
0.22617884324498036
0.5457285053655045
0.3541133662027086
0.3922077575721074
0.8850909325319567
0.4260168921791697
0.2915473523958973
0.6405928669253705
0.408404680554137
0.46825540616075173
0.8804971727488957
0.577371700359053
0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\sklearn\model_selection\_validation.py:552: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\sklearn\linear_model\_logistic.py", line 1417, in fit
    for class_, warm_start_coef_ in zip(classes_, warm_start_coef))
  File "C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\joblib\parallel.py", line 1048, in __call__
    if self.dispatch_one_batch(iterator):
  File "C:\Users\pcsea\Anaconda3\envs\study\lib\site-packages\joblib\parallel.py", line 866, in dispatch_one_batch
    self._dispatch(tasks)
  

{'C': 1} 0.5639
logistict acc: 0.5483359746434231
Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:   13.7s finished


{'n_estimators': 7} 0.5405
RamdomForest acc: 0.48335974643423135
